In [ ]:
pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
pip install torch torchvision

In [ ]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
pip install trl datasets

In [ ]:
import trl
import transformers
from datasets import load_dataset

In [ ]:
data = load_dataset("imdb", split="train")

In [ ]:
data

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
data[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
data[0].get("text")

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [ ]:
data[0].get("label")

0

In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

In [ ]:
model , tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length=2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2024.12.11: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='unsloth/mistral-7b-bnb-4bit', vocab_size=32000, model_max_length=32768, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = 2048,
)

In [ ]:
import torch

In [ ]:
args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 10,
    max_steps = 60,
    logging_steps = 1,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    learning_rate = 2e-4,
    output_dir = "outputs",
    optim = "adamw_8bit",
    seed = 3407,
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    args = args,
    max_seq_length = 2048,
    train_dataset = data,
    dataset_text_field = "text",
)

In [ ]:
import wandb

# Increase the timeout for wandb.init()
wandb.init(
    project="Finetuning",
    entity="abeshith-dr-m-g-r-educational-and-research-institute",
    settings=wandb.Settings(mode="offline", init_timeout=120),  # Increased timeout to 120 seconds
)

# Your training code here
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.436200
2,2.347200
3,2.386700
4,2.449500
5,2.581200
6,2.715600
7,2.300700
8,2.313300
9,2.194300
10,2.436500


TrainOutput(global_step=60, training_loss=2.3808915853500365, metrics={'train_runtime': 669.4384, 'train_samples_per_second': 0.717, 'train_steps_per_second': 0.09, 'total_flos': 9642624560529408.0, 'train_loss': 2.3808915853500365, 'epoch': 0.0192})

In [ ]:
review  = "i really liked the movies"

In [ ]:
input = tokenizer([
    review
  ],
  return_tensors = "pt",
  padding = True,
    ).to("cuda")

In [ ]:
input

{'input_ids': tensor([[    1,   613,  1528,  8232,   272, 10615]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
model = FastLanguageModel.for_inference(model)

In [ ]:
output = model.generate(**input,max_new_tokens=128,use_cache = True)

In [ ]:
output

tensor([[    1,   613,  1528,  8232,   272, 10615, 28725,   562,   613,  1073,
           272,  9781,  3518,   349,  1873, 28723,   272,  9781,  3518,   349,
           680,   737,   272,  3493, 19607,  1820, 28723,   272,  9781,  3518,
           349,   680,   737,   272,  3493, 19607,  1820, 28723,   272,  9781,
          3518,   349,   680,   737,   272,  3493, 19607,  1820, 28723,   272,
          9781,  3518,   349,   680,   737,   272,  3493, 19607,  1820, 28723,
           272,  9781,  3518,   349,   680,   737,   272,  3493, 19607,  1820,
         28723,   272,  9781,  3518,   349,   680,   737,   272,  3493, 19607,
          1820, 28723,   272,  9781,  3518,   349,   680,   737,   272,  3493,
         19607,  1820, 28723,   272,  9781,  3518,   349,   680,   737,   272,
          3493, 19607,  1820, 28723,   272,  9781,  3518,   349,   680,   737,
           272,  3493, 19607,  1820, 28723,   272,  9781,  3518,   349,   680,
           737,   272,  3493, 19607,  1820, 28723,  

In [ ]:
tokenizer.batch_decode(output)

['<s> i really liked the movies, but i think the tv series is better. the tv series is more like the original comic book. the tv series is more like the original comic book. the tv series is more like the original comic book. the tv series is more like the original comic book. the tv series is more like the original comic book. the tv series is more like the original comic book. the tv series is more like the original comic book. the tv series is more like the original comic book. the tv series is more like the original comic book. the tv series is more like the original comic book. the tv series is more like the original']